In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Bibliotecas importadas correctamente')

## 1. Carga y Exploración de Datos

In [ ]:
# Cargar datos
df = pd.read_csv('../datasets/desarrollo_larval_crustaceos.csv')

print('Dimensiones del dataset:', df.shape)
print('\nPrimeras filas:')
df.head(10)

In [ ]:
# Información del dataset
print('Información del dataset:')
df.info()

print('\nEstadísticas descriptivas:')
df.describe()

In [ ]:
# Distribución de estadios larvales
print('Distribución de estadios larvales:')
print(df['estadio_larval'].value_counts().sort_index())

plt.figure(figsize=(10, 5))
df['estadio_larval'].value_counts().sort_index().plot(kind='bar', color='steelblue', alpha=0.8)
plt.title('Distribución de Estadios Larvales', fontsize=14, fontweight='bold')
plt.xlabel('Estadio Larval', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Matriz de correlación
correlation_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Matriz de Correlación', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Relación entre días de cultivo y estadio larval
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

features_to_plot = ['temperatura_C', 'salinidad_ppt', 'pH', 'oxigeno_mg_L', 'densidad_larvas_m3', 'alimento_mg_dia']

for idx, feature in enumerate(features_to_plot):
    axes[idx].scatter(df[feature], df['estadio_larval'], alpha=0.6, s=50, c=df['dias_cultivo'], cmap='viridis')
    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].set_ylabel('Estadio Larval', fontsize=11)
    axes[idx].set_title(f'{feature} vs Estadio Larval', fontsize=12, fontweight='bold')
    axes[idx].grid(alpha=0.3)
    
plt.colorbar(axes[0].collections[0], ax=axes, label='Días de Cultivo', orientation='horizontal', 
             pad=0.05, aspect=40)
plt.tight_layout()
plt.show()

In [ ]:
# Distribución de variables por estadio larval
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

df.boxplot(column='temperatura_C', by='estadio_larval', ax=axes[0], patch_artist=True)
axes[0].set_title('Temperatura por Estadio', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Estadio Larval')
axes[0].set_ylabel('Temperatura (°C)')

df.boxplot(column='oxigeno_mg_L', by='estadio_larval', ax=axes[1], patch_artist=True)
axes[1].set_title('Oxígeno Disuelto por Estadio', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Estadio Larval')
axes[1].set_ylabel('Oxígeno (mg/L)')

df.boxplot(column='alimento_mg_dia', by='estadio_larval', ax=axes[2], patch_artist=True)
axes[2].set_title('Alimentación por Estadio', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Estadio Larval')
axes[2].set_ylabel('Alimento (mg/día)')

df.boxplot(column='dias_cultivo', by='estadio_larval', ax=axes[3], patch_artist=True)
axes[3].set_title('Días de Cultivo por Estadio', fontsize=12, fontweight='bold')
axes[3].set_xlabel('Estadio Larval')
axes[3].set_ylabel('Días de Cultivo')

plt.suptitle('')
plt.tight_layout()
plt.show()

## 3. Preprocesamiento de Datos

In [ ]:
# Seleccionar características
features = ['temperatura_C', 'salinidad_ppt', 'pH', 'oxigeno_mg_L', 
            'densidad_larvas_m3', 'alimento_mg_dia', 'dias_cultivo']

X = df[features]
y = df['estadio_larval']

print('Forma de X:', X.shape)
print('Forma de y:', y.shape)
print('\nRango de estadios larvales:', y.min(), '-', y.max())

In [ ]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Datos de entrenamiento:', X_train.shape)
print('Datos de prueba:', X_test.shape)

In [ ]:
# Estandarizar características (opcional para Random Forest, pero puede mejorar interpretabilidad)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✓ Datos estandarizados')

## 4. Entrenamiento del Modelo Random Forest

In [ ]:
# Crear modelo base
rf_base = RandomForestRegressor(random_state=42, n_estimators=100)
rf_base.fit(X_train, y_train)

# Predicciones iniciales
y_pred_train_base = rf_base.predict(X_train)
y_pred_test_base = rf_base.predict(X_test)

print('✓ Modelo base entrenado')
print('\nRendimiento del Modelo Base:')
print(f'R² Entrenamiento: {r2_score(y_train, y_pred_train_base):.4f}')
print(f'R² Prueba: {r2_score(y_test, y_pred_test_base):.4f}')
print(f'RMSE Prueba: {np.sqrt(mean_squared_error(y_test, y_pred_test_base)):.4f}')
print(f'MAE Prueba: {mean_absolute_error(y_test, y_pred_test_base):.4f}')

In [ ]:
# Optimización de hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

print('Buscando mejores hiperparámetros...')
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), 
                           param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print('\n✓ Búsqueda completada')
print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Mejor puntuación CV: {grid_search.best_score_:.4f}')

In [ ]:
# Entrenar modelo optimizado
rf_optimized = grid_search.best_estimator_

# Predicciones con modelo optimizado
y_pred_train = rf_optimized.predict(X_train)
y_pred_test = rf_optimized.predict(X_test)

print('Rendimiento del Modelo Optimizado:')
print(f'R² Entrenamiento: {r2_score(y_train, y_pred_train):.4f}')
print(f'R² Prueba: {r2_score(y_test, y_pred_test):.4f}')
print(f'RMSE Prueba: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.4f}')
print(f'MAE Prueba: {mean_absolute_error(y_test, y_pred_test):.4f}')

## 5. Análisis de Importancia de Características

In [ ]:
# Importancia de características
feature_importance = pd.DataFrame({
    'Característica': features,
    'Importancia': rf_optimized.feature_importances_
}).sort_values('Importancia', ascending=False)

print('Importancia de Características:')
print(feature_importance)

# Visualización
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Característica'], feature_importance['Importancia'], color='teal', alpha=0.7)
plt.xlabel('Importancia', fontsize=12)
plt.title('Importancia de Características en Random Forest', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 6. Evaluación del Modelo

In [ ]:
# Gráfico de predicciones vs valores reales
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Entrenamiento
axes[0].scatter(y_train, y_pred_train, alpha=0.6, s=50, color='blue', edgecolor='black', linewidth=0.5)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2, label='Predicción Perfecta')
axes[0].set_xlabel('Estadio Real', fontsize=12)
axes[0].set_ylabel('Estadio Predicho', fontsize=12)
axes[0].set_title('Predicciones en Entrenamiento', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Prueba
axes[1].scatter(y_test, y_pred_test, alpha=0.6, s=50, color='green', edgecolor='black', linewidth=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Predicción Perfecta')
axes[1].set_xlabel('Estadio Real', fontsize=12)
axes[1].set_ylabel('Estadio Predicho', fontsize=12)
axes[1].set_title('Predicciones en Prueba', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de residuos
residuals_test = y_test - y_pred_test

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de residuos
axes[0].scatter(y_pred_test, residuals_test, alpha=0.6, s=50, color='purple', edgecolor='black', linewidth=0.5)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicción', fontsize=12)
axes[0].set_ylabel('Residuo (Real - Predicho)', fontsize=12)
axes[0].set_title('Gráfico de Residuos', fontsize=13, fontweight='bold')
axes[0].grid(alpha=0.3)

# Distribución de residuos
axes[1].hist(residuals_test, bins=15, color='orange', alpha=0.7, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuo', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribución de Residuos', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f'Media de residuos: {residuals_test.mean():.4f}')
print(f'Desviación estándar de residuos: {residuals_test.std():.4f}')

In [ ]:
# Validación cruzada
cv_scores = cross_val_score(rf_optimized, X, y, cv=5, scoring='r2')

print('Validación Cruzada (5-fold):')
print(f'R² scores: {cv_scores}')
print(f'R² medio: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})')

## 7. Predicción en Nuevos Datos

In [ ]:
# Ejemplo de predicción para nuevas larvas
nuevas_larvas = pd.DataFrame({
    'temperatura_C': [16.5, 18.0, 19.5],
    'salinidad_ppt': [34.0, 34.5, 35.0],
    'pH': [8.1, 8.2, 8.3],
    'oxigeno_mg_L': [8.0, 8.2, 8.4],
    'densidad_larvas_m3': [45, 38, 32],
    'alimento_mg_dia': [140, 170, 200],
    'dias_cultivo': [8, 12, 16]
})

predicciones = rf_optimized.predict(nuevas_larvas)

print('🎯 Predicciones de Estadio Larval para Nuevas Muestras:')
print('=' * 70)
for i, pred in enumerate(predicciones):
    print(f'\nMuestra {i+1}:')
    print(f'  Días de cultivo: {nuevas_larvas.iloc[i]["dias_cultivo"]}')
    print(f'  Temperatura: {nuevas_larvas.iloc[i]["temperatura_C"]}°C')
    print(f'  Alimento: {nuevas_larvas.iloc[i]["alimento_mg_dia"]} mg/día')
    print(f'  → Estadio predicho: {pred:.2f} (≈ Estadio {round(pred)})')
print('=' * 70)

## 8. Análisis de Dependencia Parcial

In [ ]:
# Análisis de dependencia parcial para variables clave
from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(figsize=(14, 4))
features_to_plot = [6, 0, 5]  # dias_cultivo, temperatura_C, alimento_mg_dia

PartialDependenceDisplay.from_estimator(
    rf_optimized, X_train, features_to_plot,
    feature_names=features, ax=ax, n_cols=3
)
plt.suptitle('Gráficos de Dependencia Parcial', fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

## 9. Conclusiones

### Resultados del Modelo
- El modelo Random Forest logra predecir el estadio larval con alta precisión
- Las variables más importantes son: días de cultivo, temperatura y cantidad de alimento
- El R² indica que el modelo explica la mayor parte de la variabilidad en los datos

### Aplicaciones Prácticas
- **Gestión de cultivo**: Predecir cuándo las larvas alcanzarán estadios específicos
- **Optimización de alimentación**: Ajustar la cantidad de alimento según el estadio predicho
- **Control de calidad**: Identificar desviaciones del desarrollo esperado
- **Planificación**: Estimar tiempos de cosecha y necesidades de infraestructura

### Insights Biológicos
- La temperatura muestra una relación positiva con el desarrollo larval
- El oxígeno disuelto es crítico para el metabolismo y crecimiento
- La alimentación adecuada acelera el paso entre estadios

### Limitaciones
- Los datos sintéticos son ilustrativos; se requieren datos reales de cultivos comerciales
- El modelo no considera mortalidad ni calidad de las larvas
- Variables como luz, fotoperíodo y calidad del alimento no están incluidas

### Próximos Pasos
1. Incorporar datos reales de criaderos de crustáceos
2. Incluir variables adicionales (fotoperíodo, corrientes, composición del alimento)
3. Desarrollar modelos separados para diferentes especies
4. Implementar sistema de monitoreo en tiempo real
5. Integrar con sistemas de control automático de cultivo